# Kiểm tra giữa kỳ

## Chương 14: Kiểm định phi tham số

### 14.1 Kiểm định dấu trường hợp mẫu 

In [3]:
import numpy as np
import math
from scipy import stats

In [28]:
def stats_median(vec: np.ndarray, m0: float, level: float, kind: str = '!=') -> tuple[float, str, str]:
    sign = vec - m0
    sign = sign[sign != 0]
    sign = sign > 0
    n = np.size(sign)
    r = sign.sum()
    p_value = sum(math.comb(n, x) * (0.5 ** x) * ((1 - 0.5) ** (n - x)) for x in range(r, n + 1))

    if kind == '<':
        p_value = 1 - p_value
    elif kind == '>':
        p_value = p_value
    else:
        p_value = 2 * (1 - p_value) if r < n / 2 else 2 * p_value

    return p_value, f'm {kind} {m0}', 'Reject' if p_value < level else 'Not reject'


def stats_median_with_norm(vec: np.ndarray, m0: float, level: float, kind: str = '!=') -> tuple[float, str, str]:
    sign = vec - m0
    sign = sign[sign != 0]
    sign = sign > 0
    n = np.size(sign)
    r = sign.sum()
    z = (r - 0.5 * n) / (0.5 * math.sqrt(n))
    p_value = stats.norm.cdf(z)

    if kind == '<':
        p_value = p_value
    elif kind == '>':
        p_value = 1 - p_value
    else:
        p_value = 2 * (1 - stats.norm.cdf(abs(z)))

    return p_value, f'm {kind} {m0}', 'Reject' if p_value < level else 'Not reject'


def test(df: np.ndarray, muy0: float, level: float, kind: str = '!=') -> None:
    temp = df - muy0
    temp = temp[temp != 0]
    norm = np.size(temp) >= 10
    print(stats_median(df, muy0, level, kind))
    if norm:
        print(f'With normal distribution: {stats_median_with_norm(df, muy0, level, kind)}')

#### Bài tập 14.1 
Ten samples were taken from a plating bath used in an electronics
manufacturing process, and the bath pH of the bath was determined. The sample pH
values are 7.91, 7.85, 6.82, 8.01, 7.46, 6.95, 7.05, 7.35, 7.25, and 7.42. Manufacturing
engineering believes that pH has a median value of 7.0.
a. Do the sample data indicate that this statement is correct? Use the sign test with
α = 0.05 to investigate this hypothesis. Find the p-value for this test.
b. Use the normal approximation for the sign test to test H0 : µ = 0.7 versus
H1 : µ != 0.7. What is the p-value for this test?

In [29]:
data = np.array([7.91, 7.85, 6.82, 8.01, 7.46, 6.95, 7.05, 7.35, 7.25, 7.42])
muy = 7.0
alpha = 0.05
test(data, muy, alpha)

(0.109375, 'm != 7.0', 'Not reject')
With normal distribution: (0.05777957112359733, 'm != 7.0', 'Not reject')


#### Bài tập 14.2 
The titanium content in an aircraft-grade alloy is an important determinant of strength. A sample of 20 test coupons reveals the following titanium content
(in percent): 8.32, 8.05, 8.93, 8.65, 8.25, 8.46, 8.52, 8.35, 8.36, 8.41, 8.42, 8.30, 8.71,
8.75, 8.60, 8.83, 8.50, 8.38, 8.29, 8.46 The median titanium content should be 8.5%.
a. Use the sign test with α = 0.05 to investigate this hypothesis. Find the p-value
for this test.
b. Use the normal approximation for the sign test to test H0 : µ = 8.5 versus
H1 : µ 6= 8.5 with α = 0.05. What is the p-value for this test?

In [30]:
data = np.array([8.32, 8.05, 8.93, 8.65, 8.25, 8.46, 8.52, 8.35, 8.36, 8.41, 8.42, 8.30, 8.71,
                 8.75, 8.60, 8.83, 8.50, 8.38, 8.29, 8.46])
muy = 8.5
alpha = 0.05
test(data, muy, alpha)

(0.1670684814453125, 'm != 8.5', 'Not reject')
With normal distribution: (0.25134910881022265, 'm != 8.5', 'Not reject')


#### Bài tập 14.3 
The impurity level (in ppm) is routinely measured in an intermediate
chemical product. The following data were observed in a recent test: 2.4, 2.5, 1.7, 1.6,
1.9, 2.6, 1.3, 1.9, 2.0, 2.5, 2.6, 2.3, 2.0, 1.8, 1.3, 1.7, 2.0, 1.9, 2.3, 1.9, 2.4, 1.6 Can you
claim that the median impurity level is less than 2.5 ppm?
a. State and test the appropriate hypothesis using the sign test with α = 0.05. What
is the p-value for this test?
b. Use the normal approximation for the sign test to test H0 : µ = 2.5 versus
H1 : µ < 2.5 with α = 0.05. What is the p-value for this test?

In [31]:
data = np.array([2.4, 2.5, 1.7, 1.6, 1.9, 2.6, 1.3, 1.9, 2.0, 2.5, 2.6,
                 2.3, 2.0, 1.8, 1.3, 1.7, 2.0, 1.9, 2.3, 1.9, 2.4, 1.6])
muy = 2.5
alpha = 0.05
test(data, muy, alpha, '<')

(2.002716064453125e-05, 'm < 2.5', 'Reject')
With normal distribution: (0.0001733096755673334, 'm < 2.5', 'Reject')


### 14.2 Kiểm định dấu - hạng Wilcoxon trường hợp mẫu cặp

In [55]:
def wilcoxon_1_sample(df: np.ndarray, muy0: float) -> int:
    table = []
    for x in df:
        if x - muy0 != 0:
            temp = x - muy0
            table.append((temp, temp > 0))

    minus_w = 0
    plus_w = 0
    sort_table = sorted(table, key=lambda v: abs(v[0]))
    for i, x in enumerate(sort_table):
        if x[1]:
            plus_w += i + 1
        else:
            minus_w += i + 1

    return min(minus_w, plus_w)


def wilcoxon_2_sample(df1: np.ndarray, df2: np.ndarray, diff: float = 0) -> int:
    table = []
    for x, y in zip(df1, df2):
        if x - y - diff != 0:
            temp = x - y
            table.append((temp, temp > 0))

    minus_w = 0
    plus_w = 0
    sort_table = sorted(table, key=lambda v: abs(v[0]))
    for i, x in enumerate(sort_table):
        if x[1]:
            plus_w += i + 1
        else:
            minus_w += i + 1

    return min(minus_w, plus_w)

#### Bài tập 14.4 
An inspector are measured the diameter of a ball bearing using a new
type of caliper. The results were as follows (in mm): 0.265, 0.263, 0.266, 0.267, 0.267,
0.265, 0.267,0.267, 0.265, 0.268, 0.268, and 0.263.
a. Use the Wilcoxon signed-rank test to evaluate the claim that the mean ball diameter is 0.265 mm. Use α = 0.05.
b. Use the normal approximation for the test. With α = 0.05, what conclusions can
you draw?

In [51]:
data = np.array([0.265, 0.263, 0.266, 0.267, 0.267, 0.265,
                 0.267, 0.267, 0.265, 0.268, 0.268, 0.263])
muy = 0.265
alpha = 0.05
print(wilcoxon_1_sample(data, muy))
test(data, muy, alpha)

9
(0.1796875, 'm != 0.265', 'Not reject')


#### Bài tập 14.5 
A new type of tip can be used in a Rockwell hardness tester. Eight
coupons from test ingots of a nickelbased alloy are selected, and each coupon is tested
using the new tip. The Rockwell C-scale hardness readings are 63, 65, 58, 60, 55, 57,
53, and 59. Do the results support the claim that the mean hardness exceeds 60 at a
0.05 level?

In [52]:
data = np.array([63, 65, 58, 60, 55, 57, 53, 59])
muy = 60
wilcoxon_1_sample(data, muy)

8

#### Bài tập 14.6 
A primer paint can be used on aluminum panels. The primer’s drying
time is an important consideration in the manufacturing process. Twenty panels are
selected, and the drying times are as follows: 1.6, 1.3, 1.5, 1.6, 1.7, 1.9, 1.8, 1.6, 1.4,
1.8, 1.9, 1.8, 1.7, 1.5, 1.6, 1.4, 1.3, 1.6, 1.5, and 1.8. Is there evidence that the mean
drying time of the primer exceeds 1.5 hr?

In [53]:
data = np.array([1.6, 1.3, 1.5, 1.6, 1.7, 1.9, 1.8, 1.6, 1.4, 1.8,
                 1.9, 1.8, 1.7, 1.5, 1.6, 1.4, 1.3, 1.6, 1.5, 1.8])
muy = 1.5
wilcoxon_1_sample(data, muy)

29

#### Bài tập 14.7 
A random sample of eight pairs of twins were randomly assigned to
treatment A or treatment B. The data are given in the following table.
Is there significant evidence that the two treatments differ using an a α = 0.05 Wilcoxon
signed-rank test.

In [56]:
data1 = np.array([48.3, 44.6, 49.7, 40.5, 54.3, 55.6, 45.8, 35.4])
data2 = np.array([43.5, 43, 8, 53.7, 43.9, 54.4, 54.7, 45.2, 34.4])
wilcoxon_2_sample(data1, data2)

16

#### Bài tập 14.8 
A study was conducted to determine whether automobile repair charges
are higher for female customers than for male customers. Twenty auto repair shops
were randomly selected from the telephone book. Two cars of the same age, brand,
and engine problem were used in the study. For each repair shop, the two cars were
randomly assigned to a man and woman participant and then taken to the shop for an
estimate of repair cost. The repair costs (in dollars) are given here.
a. Which procedure, t or Wilcoxon, is more appropriate in this situation? Why?
b. Are repair costs generally higher for female customers than for male customers?
Use a α = 0.05.

In [57]:
data1 = np.array(
    [871, 648, 795, 838, 1033, 917, 1047, 723, 1179, 707, 817, 846, 975, 868, 1323, 791, 1157, 932, 1089, 770])
data2 = np.array([792, 765, 611, 520, 618, 447, 548, 720, 899, 788, 927, 657, 851, 702, 918, 528, 884, 702, 839, 878])
wilcoxon_2_sample(data1, data2)

18

#### Bài tập 14.9 
The effect of Benzedrine on the heart rate of dogs (in beats per minute)
was examined in an experiment on 14 dogs chosen for the study. Each dog was to serve
as its own control, with half of the dogs assigned to receive Benzedrine during the first
study period and the other half assigned to receive a placebo (saline solution). All dogs
were examined to determine the heart rates after 2 hours on the medication. After 2
weeks in which no medication was given, the regimens for the dogs were switched for
the second study period. The dogs previously on Benzedrine were given the placebo,
and the others received Benzedrine. Again, heart rates were measured after 2 hours.
The following sample data are not arranged in the order in which they were taken but
have been summarized by regimen. Use these data to test the research hypothesis that
the distribution of heart rates for the dogs when receiving Benzedrine is shifted to the
right of that for the same animals when on the placebo. Use a one-tailed Wilcoxon
signed-rank test with a α = 0.05.

In [58]:
data1 = np.array([250, 271, 243, 252, 266, 272, 293, 296, 301, 298, 310, 286, 306, 309])
data2 = np.array([258, 285, 245, 250, 268, 278, 280, 305, 319, 308, 320, 293, 305, 313])
wilcoxon_2_sample(data1, data2)

16

### 14.3 Kiểm định Mann-Whitney trường hợp mẫu độc lập

In [63]:
def mann_whitney(df1: np.ndarray, df2: np.ndarray) -> dict[str, int]:
    if np.size(df1) > np.size(df2):
        df1, df2 = df2, df1

    n1 = np.size(df1)
    n2 = np.size(df2)

    merge = np.sort(np.concatenate((df1, df2)))
    w1 = 0
    for i, v in enumerate(merge):
        if v in df1:
            w1 += i

    w2 = int((n1 + n2) * (n1 + n2 + 1) / 2 - w1)
    return {'W1': w1, 'W2': w2}

#### Bài tập 14.10 
An electrical engineer must design a circuit to deliver the maximum
amount of current to a display tube to achieve sufficient image brightness. Within
her allowable design constraints, she has developed two candidate circuits and tests
prototypes of each. The resulting data (in microamperes) are as follows:
a. Use the Wilcoxon rank-sum test to test H0 : µ1 = µ2 the alternative H1 : µ1 > µ2. Use α = 0.025.
b. Use the normal approximation for the Wilcoxon rank-sum test. Assume that α = 0.05. Find the approximate p-value for this test statistic.

In [66]:
data1 = np.array([251, 255, 258, 257, 250, 251, 254, 250, 248])
data2 = np.array([250, 253, 249, 256, 259, 252, 260, 251])
mann_whitney(data1, data2)

{'W1': 88, 'W2': 65}

#### Bài tập 14.11 
One of the authors travels regularly to Seattle, Washington. He uses
either Delta or Alaska airline. Flight delays are sometimes unavoidable, but he would
be willing to give most of his business to the airline with the best on-time arrival record.
The number of minutes that his flight arrived late for the last six trips on each airline
follows. Is there evidence that either airline has superior on-time arrival performance?
Use α = 0.01 and the Wilcoxon rank-sum test

In [67]:
data1 = np.array([13, 10, 1, -4, 0, 9])
data2 = np.array([15, 8, 3, -1, -2, 4])
mann_whitney(data1, data2)

{'W1': 34, 'W2': 44}

#### Bài tập 14.12 
The manufacturer of a hot tub is interested in testing two different
heating elements for its product. The element that produces the maximum heat gain
after 15 minutes would be preferable. The manufacturer obtains 10 samples of each
heating unit and tests each one. The heat gain after 15 minutes (in ◦F) follows.
a. Is there any reason to suspect that one unit is superior to the other? Use α = 0.05
and the Wilcoxon ranksum test.
b. Use the normal approximation for the Wilcoxon ranksum test. Assume that α =
0.05. What is the approximate p-value for this test statistic?

In [68]:
data1 = np.array([25, 27, 29, 31, 30, 26, 24, 32, 33, 38])
data2 = np.array([31, 33, 32, 35, 34, 29, 38, 35, 37, 30])
mann_whitney(data1, data2)

{'W1': 128, 'W2': 82}

#### Bài tập 14.13 
Random samples of size n1 = 8 and n2 = 8 were selected from populations A and B, respectively. The data are given in the following table.
a. Test for a difference in the medians of the two populations using an a α = 0.05
Wilcoxon rank sum test.
b. Place a 95% confidence interval on the difference in the medians of the two populations.

In [69]:
data1 = np.array([4.3, 4.6, 4.7, 5.1, 5.3, 5.3, 5.8, 5.4])
data2 = np.array([3.5, 3.8, 3.7, 3.9, 4.4, 4.7, 5.2, 4.4])
mann_whitney(data1, data2)

{'W1': 92, 'W2': 44}